Martín Amado - 19020
Juan Pablo Pineda - 19087

Referencia para análisis de datos: https://www.kaggle.com/competitions/digit-recognizer/data?select=test.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from scipy import stats
import matplotlib.cm as cm
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from factor_analyzer import calculate_bartlett_sphericity, calculate_kmo

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from apyori import apriori

# Ejercicio 1

# Ejercicio 2

## Parte 1: Series de tiempo

### Consumos Diesel

Recaudacion y limpieza de datos

In [ ]:
consumption = pd.read_excel('./CONSUMO-2022-05.xlsx', skiprows=6)
consumption = consumption[['Fecha', 'Diesel']]
#omitimos los datos despues del 269 ya que no aportan relevancia
consumption = consumption[:269]

Desarrollo de la seria de tiempo

In [ ]:
trainSize = int(len(consumption) * 0.7)
trainConsumos = consumption[0:trainSize]
testConsumos = consumption[trainSize:len(consumption)]
trainConsumos = trainConsumos.set_index(['Fecha'])
testConsumos = testConsumos.set_index(['Fecha'])

In [ ]:
tsDiesel = trainConsumos['Diesel']
mediaMovil = tsDiesel.rolling(window=12).mean()
deMovil = tsDiesel.rolling(window=12).std()

In [ ]:
original = plt.plot(tsDiesel, color="blue", label="Original")
media = plt.plot(mediaMovil, color='red', label = 'Media Movil')
ds = plt.plot(deMovil,color='black', label = 'Desviación Estándar Móvil')
plt.legend(loc = 'best')
plt.title('Media y desviación estándar móvil')
plt.show(block=False)

In [ ]:
descomposicion = seasonal_decompose(tsDiesel)
descomposicion.plot()

In [ ]:
tsDiesel = tsDiesel.astype({'Diesel':'float'})
tsDieselLog = np.log(tsDiesel)
plt.plot(tsDieselLog)

In [ ]:
dfTest = adfuller(tsDiesel, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Valor Crítico (%s)'%key] = value
print(salidaDf)

In [ ]:
tsDieselDiff = tsDiesel.diff()
tsDieselDiff.fillna(0,inplace=True)
dfTest = adfuller(tsDieselDiff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
salidaDf

In [ ]:
plt.plot(tsDieselDiff)
plt.title('Diferenciacion de la serie')

In [ ]:
modelo111 = SARIMAX(tsDieselLog, order=(1,1,1), seasonal_order=(3,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m111 = modelo111.fit()
resultado_m111.summary().tables[1]

In [ ]:
resultado_m111.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
testConsumos.index[0]
pred = resultado_m111.get_prediction(start=testConsumos.index[0], dynamic=False)
pred_ci = pred.conf_int()
consumoIndexed = consumption.set_index('Fecha')
consumoIndexed = consumoIndexed['Diesel']
ax = consumoIndexed['2000':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between( pred_ci.iloc[:,0],
                pred_ci.iloc[:,1], color='k', alpha=.2)
#ax.set_xlabel('Date')
#ax.set_ylabel('Retail_sold')
plt.legend()
plt.show()

pred

## Parte 2: LSTM

In [ ]:
consumo = pd.read_excel('./CONSUMO-2022-05.xlsx', skiprows=6)
consumo = consumo[['Fecha', 'Diesel']]
#omitimos los datos despues del 269 ya que no aportan relevancia
consumo = consumo[:269]
consumo = consumo.set_index('Fecha')
consumo.head()

In [ ]:
plt.plot(consumo)
plt.gca().set(title="Consumo de Diesel por mes por año", xlabel="Fecha", ylabel="Consumo")
plt.show()

Estacionarizar para uso de LSTM

In [ ]:
# Se calcula la media móvil y la desviación estandar móvil de los últimos 12 meses.
mediaMovil = consumo.rolling(window=12).mean()
deMovil = consumo.rolling(window=12).std()
# Se grafican los resultados.
original = plt.plot(consumo, color="blue", label="Original")
media = plt.plot(mediaMovil, color='red', label = 'Media Movil')
ds = plt.plot(deMovil,color='black', label = 'Desviación Estándar Móvil')
plt.legend(loc = 'best')
plt.title('Media y desviación estándar móvil')
plt.show(block=False)

In [ ]:
descomposicion = seasonal_decompose(consumo)
descomposicion.plot()

Existe tendencia

In [ ]:
print('Resultados del Test de Dickey Fuller')
dfTest = adfuller(consumo, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
print('Resultados del Test de Dickey Fuller para una diferenciación de la serie')
diesel_diff = consumo.diff()
diesel_diff.fillna(0,inplace=True)
dfTest = adfuller(diesel_diff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
plt.plot(diesel_diff)

Necesidad de diferenciacion

In [ ]:
scaler = StandardScaler()
diesel_scaled = scaler.fit_transform(diesel_diff) 
diesel_scaled[1:7]

60% entrenamiento, 20% validacion y prueba

In [ ]:
train_diesel = round(0.6*len(diesel_scaled))
val_test_diesel = round(0.2*len(diesel_scaled))
test = diesel_scaled[(train_diesel + val_test_diesel) - 1:]
validation = diesel_scaled[(train_diesel):train_diesel + val_test_diesel + 1]
train = diesel_scaled[0:train_diesel]
train = np.insert(train, 0, 0)
train = np.reshape(train, (train.shape[0], 1))

Transformacion a serie supervisada

In [ ]:
from operator import concat


def supervisada(serie,retrasos = 1):
    serie_x = []
    serie_y = []
    for i in range(len(serie)-retrasos):
        valor = serie[i:(i+retrasos),0]
        valor_sig = serie[i+retrasos,0]
        serie_x.append(valor)
        serie_y.append(valor_sig)
    return np.array(serie_x), np.array(serie_y)

x_train,y_train = supervisada(train)
x_val,y_val = supervisada(validation)
x_test,y_test = supervisada(test)

#### Creacion de modelo

##### Modelo 1

Matrices de 3 dimensiones

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0],1,1))
x_val = np.reshape(x_val, (x_val.shape[0],1,1))
x_test = np.reshape(x_test, (x_test.shape[0],1,1))

Modelo con una capa LSTM

In [ ]:
modelo1 = Sequential()
lote = 1
unidades =  1
paso = 1
caracteristicas = 1 #es univariada
modelo1.add(LSTM(lote, batch_input_shape=(lote,paso,caracteristicas),stateful=True))
modelo1.add(Dense(1))
modelo1.summary()

In [ ]:
modelo1.compile(loss='mean_squared_error',optimizer="rmsprop")

Entrenamiento

In [ ]:
epocas = 50
history= modelo1.fit(
    x = x_train,
    y = y_train,
    batch_size = lote,
    epochs = epocas,
    shuffle = False,
    validation_data = (x_val,y_val),
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='center')

In [ ]:
print("Pérdida en Entrenamiento")
modelo1.evaluate(
    x = x_train,
    y = y_train
)
print("Pérdida en Validación")
modelo1.evaluate(
    x = x_val,
    y = y_val
)
print("Pérdida en Prueba")
modelo1.evaluate(
    x = x_test,
    y = y_test
)

Prediccion modelo 1

In [ ]:
prediccion_val = []

def prediccion_fun(data,modelo, batch_size,scaler,dif=False,dif_cant=1, Series = None , n=1):
    prediccion = [0]* (len(data))
    i=0
    for X in data:
        X = np.reshape(X,(1,1,1))
        yhat = modelo1.predict(X, batch_size=batch_size,verbose=0)
        # invert scaling
        yhat = scaler.inverse_transform(yhat)
        if dif:
             # invert differencing
            yhat  = yhat + Series[(n+dif_cant*i)]
        # store
        prediccion[i]=yhat[0][0]
        i = i+1
    return prediccion
prediccion_val = prediccion_fun(x_val,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel)
prediccion_test = prediccion_fun(x_test,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel+val_test_diesel)


In [ ]:
df_val = pd.DataFrame(prediccion_val,index=consumo[(train_diesel):train_diesel+val_test_diesel].index)
df_test = pd.DataFrame(prediccion_test,index=consumo[train_diesel+len(df_val):len(consumo)].index)


plt.plot(consumo)
plt.plot(df_val)
plt.plot(df_test)

##### Modelo 2

Para el modelo 2, las unidades cambiaran

In [ ]:
modelo2 = Sequential()
lote = 1
unidades =  5
paso = 1
caracteristicas = 1 #es univariada
modelo2.add(LSTM(lote, batch_input_shape=(lote,paso,caracteristicas),stateful=True))
modelo2.add(Dense(1))
modelo2.summary()

In [ ]:
modelo2.compile(loss='mean_squared_error',optimizer="rmsprop")

In [ ]:
epocas = 50
history= modelo2.fit(
    x = x_train,
    y = y_train,
    batch_size = lote,
    epochs = epocas,
    shuffle = False,
    validation_data = (x_val,y_val),
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='center')

In [ ]:
print("Pérdida en Entrenamiento")
modelo2.evaluate(
    x = x_train,
    y = y_train
)
print("Pérdida en Validación")
modelo2.evaluate(
    x = x_val,
    y = y_val
)
print("Pérdida en Prueba")
modelo2.evaluate(
    x = x_test,
    y = y_test
)

In [ ]:
prediccion_val = []

def prediccion_fun(data,modelo, batch_size,scaler,dif=False,dif_cant=1, Series = None , n=1):
    prediccion = [0]* (len(data))
    i=0
    for X in data:
        X = np.reshape(X,(1,1,1))
        yhat = modelo1.predict(X, batch_size=batch_size,verbose=0)
        # invert scaling
        yhat = scaler.inverse_transform(yhat)
        if dif:
             # invert differencing
            yhat  = yhat + Series[(n+dif_cant*i)]
        # store
        prediccion[i]=yhat[0][0]
        i = i+1
    return prediccion
prediccion_val = prediccion_fun(x_val,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel)
prediccion_test = prediccion_fun(x_test,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel+val_test_diesel)


In [ ]:
df_val = pd.DataFrame(prediccion_val,index=consumo[(train_diesel):train_diesel+val_test_diesel].index)
df_test = pd.DataFrame(prediccion_test,index=consumo[train_diesel+len(df_val):len(consumo)].index)


plt.plot(consumo)
plt.plot(df_val)
plt.plot(df_test)

